In [ ]:
'''
Написать приложение, которое собирает основные новости с сайта на выбор news.mail.ru, lenta.ru, yandex-новости. 
Для парсинга использовать XPath. Структура данных должна содержать:
название источника;
наименование новости;
ссылку на новость;
дата публикации.
Сложить собранные новости в БД
'''

In [ ]:
from lxml import html
import requests
from pprint import pprint

header = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}

url = 'https://www.lenta.ru'

session = requests.Session()
response = session.get(url, headers=header)

dom = html.fromstring(response.text)

In [118]:
# //a[@class='card-mini _longgrid']
phones = []
types_of_news_nodes = ["//a[@class='card-mini _longgrid']",
                       "//a[@class='card-mini _topnews']"]

len(items)
news_array = []
for query in types_of_news_nodes:
    items = dom.xpath(query)
    for i in items:
        news_item = {'source': 'lenta.ru', 'title':'', 'date':'', 'link':''}
        try:
            news_item['source'] = 'lenta.ru'
            news_item['title'] = i.xpath(".//span/text()")[0]
            news_item['date'] = i.xpath(".//div/time/text()")[0]
            news_item['link'] = 'https://lenta.ru' + i.xpath("./@href")[0]
            news_array.append(news_item)
            # print('https://lenta.ru' + i.xpath("./@href")[0])
            # print(i.xpath(".//span/text()")[0])
            # print(i.xpath(".//div/time/text()")[0])
        except:
            pass
    print(len(news_array))


69
81


In [120]:
news_array[:3]

[{'source': 'lenta.ru',
  'title': 'Бари Алибасов назвал главные ошибки в своей жизни',
  'date': '12:02',
  'link': 'https://lenta.ruhttps://moslenta.ru/news/lyudi/bari-alibasov-13-07-2022.htm'},
 {'source': 'lenta.ru',
  'title': 'Звезда «Во все тяжкие» рассказал об инфаркте на съемках «Лучше звоните Солу»',
  'date': '11:56',
  'link': 'https://lenta.ru/news/2022/07/13/callsoul/'},
 {'source': 'lenta.ru',
  'title': 'Посол ЛНР обвинил ВСУ в уничтожении больниц на оставленных территориях',
  'date': '11:54',
  'link': 'https://lenta.ru/news/2022/07/13/lbr_hospital/'}]

In [122]:
for id, item in enumerate(news_array):
    news_array[id]['_id'] = hash(frozenset(item.items()))
news_array[0]

{'source': 'lenta.ru',
 'title': 'Бари Алибасов назвал главные ошибки в своей жизни',
 'date': '12:02',
 'link': 'https://lenta.ruhttps://moslenta.ru/news/lyudi/bari-alibasov-13-07-2022.htm',
 '_id': 1998652689112135179}

In [125]:
###  Загружаем данные в БД

from pymongo import MongoClient
import pymongo

# Запускаем сервер MongoDB в Docker'e

# https://hub.docker.com/_/mongo
# docker pull mongo
# docker run --name some-mongo -d mongo:tag

# Подключаемся к серверу. IP адрес находим через docker inspect
# docker inspect {container_id} -> monogodb IP Address
client = MongoClient('172.17.0.2', 27017)

In [126]:

# Переключаемся на БД новостей
news_db = client['news_db']
# Создаем раздел новостей с lenta.ru
lenta_db = news_db.lentaru

In [133]:
# Загружаем новости в раздел lenta.ru
import pymongo
from pymongo import errors
for i in range(len(news_array)):
    doc = news_array[i]
    # print(doc)
    try:
        lenta_db.insert_one(doc)
    except errors.DuplicateKeyError:
        print(f"Document with hash = {doc['_id']} is already exists")

Document with hash = 1998652689112135179 is already exists
Document with hash = -1151951631188106829 is already exists
Document with hash = 607340052048478650 is already exists
Document with hash = -5197841739870923715 is already exists
Document with hash = 4161179272525698822 is already exists
Document with hash = -7154723606627117978 is already exists
Document with hash = 8793154413947728986 is already exists


In [138]:
# Читаем записи из раздела lenta.ru БД новостей
from pprint import pprint
items_in_db = []

items_in_db = lenta_db.find({})

for item in range(3):
    pprint(items_in_db[item])

{'_id': 1998652689112135179,
 'date': '12:02',
 'link': 'https://lenta.ruhttps://moslenta.ru/news/lyudi/bari-alibasov-13-07-2022.htm',
 'source': 'lenta.ru',
 'title': 'Бари Алибасов назвал главные ошибки в своей жизни'}
{'_id': -1151951631188106829,
 'date': '11:56',
 'link': 'https://lenta.ru/news/2022/07/13/callsoul/',
 'source': 'lenta.ru',
 'title': 'Звезда «Во все тяжкие» рассказал об инфаркте на съемках «Лучше '
          'звоните Солу»'}
{'_id': 607340052048478650,
 'date': '11:54',
 'link': 'https://lenta.ru/news/2022/07/13/lbr_hospital/',
 'source': 'lenta.ru',
 'title': 'Посол ЛНР обвинил ВСУ в уничтожении больниц на оставленных '
          'территориях'}
